In [ ]:
import json
import pandas as pd
import numpy as np
import ast
import math 

from nltk.parse.generate import generate, demo_grammar
from nltk import CFG

def remove_second_pair(list_orders):
    removed_second_pairs=[]
    for item in list_orders:
        removed_second_pairs.append(item[0])
    return(removed_second_pairs)


def add_words_lists(words,list_orders):
    table = pd.concat([words,list_orders],axis=1)
    new_list_orders=[]
        
    for row in range(0,len(table)):
        if (table.iloc[row,0]!=np.NaN) and (table.iloc[row,1]!=[]):            
            if table.iloc[row,0] in table.iloc[row,1]:
                table.iloc[row,1].remove(table.iloc[row,0])
            table.iloc[row,1].insert(0,table.iloc[row,0])

    return (table.iloc[:,1])
        
def convert_list_2_cfg(ordinary_list):
    acc_str=""
    for w in ordinary_list:
        acc_str+=" | "+ "\""+w+"\""
    return(acc_str)    
    

def generate_cfg(wpos,w1,w2,w3,s1,s2,s3):
    cfg_rules=[]

    for idx in range(0,len(wpos)):
        wpos[idx][1]=wpos[idx][1].replace(" ", "_")
        wpos[idx][1]=wpos[idx][1].capitalize()+str(idx+1)


    str_start="S -> "
    for idx in range(0,len(wpos)):
        str_start +=" "+wpos[idx][1]

    cfg_rules.append(str_start)  


    for idx in range(0,len(wpos)):
        str_rule=""
        str_rule +=wpos[idx][1]+" -> "+"\""+wpos[idx][0]+"\""
        cfg_rules.append(str_rule)  


    if not isinstance(w1, str):
        if np.isnan(w1):
            w1=""
            
    if not isinstance(w2, str):
        if np.isnan(w2):
            w2=""
            
    if not isinstance(w3, str):
        if np.isnan(w3):
            w3=""            

    
    cfg_rules_augmented=[]    
    for rule in cfg_rules:
        if rule.find(w1)>=0:
            cfg_rules_augmented.append(rule+convert_list_2_cfg(s1[1:]))
        elif rule.find(w2)>=0:
            cfg_rules_augmented.append(rule+convert_list_2_cfg(s2[1:]))
        elif rule.find(w3)>=0:
            cfg_rules_augmented.append(rule+convert_list_2_cfg(s3[1:]))
        else:
            cfg_rules_augmented.append(rule)


    cfg_grammar=""
    for rule in cfg_rules_augmented:
        cfg_grammar+=rule+"\n"     
    
    return(cfg_grammar)      
   

In [ ]:
import pandas as pd

word_table=pd.read_excel(
    "/Users/mehdi.mirzapour/Dropbox/PractiKPharma/PractiKPharma_Code/excels/4_Phase3_TranslationTable.xlsx")   

    

word_table=word_table.fillna("")

for row_w in range(word_table.shape[0]):
        word_table["syn_1"][row_w]=syn_table["Synonyms_Voting_Top10"][row_s]


# word_table.to_excel(
#     "/Users/mehdi.mirzapour/Dropbox/PractiKPharma/PractiKPharma_Code/excels/5_Phase3_TranslationTable.xlsx")           
    
word_table
    


In [ ]:
lexicalclues=pd.read_excel(
    "/Users/mehdi.mirzapour/Dropbox/PractiKPharma/Lexicons/FrenchTranslation_Phase2_Step_2.xlsx")

lexicalclues["CFG_Rule"]=""
lexicalclues["L(CFG_Rule)"]=""

exp_fr=lexicalclues["French Expression"]
exp_en=lexicalclues["English Expression(s)"].apply(lambda s: ast.literal_eval(s))

word_pos=lexicalclues["Tokenization with POS"].apply(lambda s: ast.literal_eval(s))


word1=lexicalclues["word1"]
syn1_list=lexicalclues["syn1_ordered"].apply(lambda s: ast.literal_eval(s))
syn1_list=syn1_list.apply(remove_second_pair)
syn1_list=add_words_lists(word1,syn1_list)

word2=lexicalclues["word2"]
syn2_list=lexicalclues["syn2_ordered"].apply(lambda s: ast.literal_eval(s))
syn2_list=syn2_list.apply(remove_second_pair)
syn2_list=add_words_lists(word2,syn2_list)

word3=lexicalclues["word3"]
syn3_list=lexicalclues["syn3_ordered"].apply(lambda s: ast.literal_eval(s))
syn3_list=syn3_list.apply(remove_second_pair)
syn3_list=add_words_lists(word3,syn3_list)



json_notebook = {}  

json_notebook["metadata"] = {}
json_notebook["metadata"]["kernelspec"]={}
json_notebook["metadata"]["kernelspec"]["display_name"]= "Python 3"
json_notebook["metadata"]["kernelspec"]["language"]= "python"
json_notebook["metadata"]["kernelspec"]["name"]= "python3"

json_notebook["metadata"]["language_info"]={}
json_notebook["metadata"]["language_info"]["codemirror_mode"]={}
json_notebook["metadata"]["language_info"]["codemirror_mode"]["name"]= "ipython"
json_notebook["metadata"]["language_info"]["codemirror_mode"]["version"]= 3

json_notebook["metadata"]["file_extension"]= ".py"
json_notebook["metadata"]["mimetype"]= "text/x-python"
json_notebook["metadata"]["name"]= "python"
json_notebook["metadata"]["nbconvert_exporter"]= "python"
json_notebook["metadata"]["pygments_lexer"]= "ipython3"
json_notebook["metadata"]["version"]= "3.7.2"


json_notebook["nbformat"]= 4
json_notebook["nbformat_minor"]=2



seperation_margin=[[0,100],[101,200],[201,300],[301,400],[401,500],[501,600],[601,735]]


for sm in seperation_margin:
    
    json_notebook["cells"]=[]
    
    index_start=sm[0]
    index_end=sm[1]
    
    strlist="_"+str(index_start)+"_"+str(index_end)
    
    threshold=15
    
    for i in range(index_start,index_end+1): 

        expfr=exp_fr[i]
        expen=exp_en[i]
        wpos=word_pos[i]
        w1=word1[i]
        w2=word2[i]
        w3=word3[i]
        s1=syn1_list[i][0:threshold]
        s2=syn2_list[i][0:threshold]
        s3=syn3_list[i][0:threshold]

        cfg_grammar=generate_cfg(wpos,w1,w2,w3,s1,s2,s3)

        
        json_cell={}
        json_cell["cell_type"] = "code"
        json_cell["metadata"] = {}
        json_cell["outputs"] = []
        json_cell["execution_count"] = "null"
        json_source1=["from nltk.parse.generate import generate, demo_grammar\n",
            "from nltk import CFG\n",
            "\n",
            "#  Item Number     : "+str(i)+"\n",
            "#  French Term     : "+str(expfr)+"\n",
            "#  English Term(s) : "+str(expen)+"\n\n",
            "\n",
            "cfg_grammar= \"\"\"\n"]
        json_source2=[cfg_grammar]
        json_source3=["\"\"\"\n",
            "\n",
            "for sentence in generate(CFG.fromstring(cfg_grammar), n=1000):\n",
            "    print(' '.join(sentence))"]
        json_cell["source"]=json_source1+json_source2+json_source3

        json_notebook["cells"].append(json_cell)



    with open("Validation_Notebook"+strlist+".ipynb", 'w') as outfile:  
        json.dump(json_notebook, outfile)
print("Finished...")
 
    
